In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings("ignore")

import nltk
from thefuzz import fuzz
from thefuzz import process

In [2]:
%%time
df=pd.read_csv('wiki_movie_plots_deduped.csv')
df['Genre'].nunique()

Wall time: 1.14 s


2265

In [3]:

df['Genre']= df['Genre'].str.replace("unknown", "")


In [4]:
dax=df['Genre'].value_counts().rename_axis('Genres').reset_index(name='counts')
dax[:30]

,Genres,counts
0,,6083
1,drama,5964
2,comedy,4379
3,horror,1167
4,action,1098
5,thriller,966
6,romance,923
7,western,865
8,crime,568
9,adventure,526


In [5]:
[i for i in dax['Genres'].to_list() if len(i.split())<2][1:18]

['drama',
 'comedy',
 'horror',
 'action',
 'thriller',
 'romance',
 'western',
 'crime',
 'adventure',
 'musical',
 'mystery',
 'war',
 'animation',
 'sci-fi',
 'family',
 'fantasy',
 'animated']

In [6]:
choices = [i for i in dax['Genres'].to_list() if len(i.split())<2][1:18]


In [7]:
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        

In [8]:
%%time
for i in range(10,len(dax['Genres'].to_list()),10):
    inputt=dax['Genres'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)


Wall time: 1min 55s


In [9]:
df['Genre']=df['Genre'].apply(lambda x: x.strip())

In [10]:
[[row['Genres'],row['counts']] for i,row in dax.iterrows() if row['Genres'] in choices ]

[['', 6083],
 ['drama', 5964],
 ['comedy', 4379],
 ['horror', 1167],
 ['action', 1098],
 ['thriller', 966],
 ['romance', 923],
 ['western', 865],
 ['crime', 568],
 ['adventure', 526],
 ['musical', 467],
 ['science fiction', 418],
 ['film noir', 345],
 ['mystery', 310],
 ['war', 273],
 ['animation', 264],
 ['sci-fi', 221],
 ['family', 217],
 ['fantasy', 204],
 ['animated', 195],
 ['biography', 136],
 ['anime', 112],
 ['suspense', 104],
 ['social', 82],
 ['historical', 77],
 ['documentary', 73],
 ['serial', 71],
 ['biopic', 57],
 ['superhero', 48],
 ['martial arts', 46],
 ['short', 30],
 [' ', 28],
 ['tokusatsu', 28],
 ['spy', 27],
 ['dramedy', 24],
 ['sports', 23],
 ['mythology', 22],
 ['biographical', 20],
 ['folklore', 17],
 ['masala', 16],
 ['rom com', 14],
 ['propaganda', 13],
 ['kaiju', 13],
 ['devotional', 12],
 ['slapstick', 12],
 ['slasher', 12],
 ['disaster film', 11],
 ['swashbuckler', 10],
 ['epic', 10],
 ['wuxia', 9],
 ['satire', 8],
 ['parody', 8],
 ['detective', 7],
 ['ant

In [11]:
replace_dict={
   "science_friction":"sci-fi",
    "sci fi":"sci-fi",
    "science friction":"sci-fi",
    "amine":"animated",
    "animation":"animated",
    "cartoon":"animated",
    "tokusatsu": "animated",
    "superhero": "animated",
    "suspense": "thriller",
    "dramedy": "drama",
    "blaxploitation": "exploitation",
    "biopic": "biography",
    "bio-pic": "biography",
    "biographical": "biography",
    "kung fu": "martial_arts",
    "martial arts": "martial_arts",
    "kung fu": "martial_arts",
    "film": "",
    "neo-noir": "noir",
    "devotional": "religious",
    "children": "teen",
    "porno": "adult",
    "history": "historical",
    "rom com": "romance-comedy",
    "rom-com": "romance-comedy",
    "love": "romance",
}

In [12]:
for key,items in replace_dict.items():
    df['Genre']= df['Genre'].str.replace(key,items)

In [13]:
dax=df['Genre'].value_counts().rename_axis('Genres').reset_index(name='counts')
choices = [i for i in dax['Genres'].to_list() if len(i.split())<2][0:18]
li=[]
def list_prep(data):
    for i in data:
        if i[1]==[]:
            choices.append(i[0])
        else:
            li.append(i[0])
        


In [14]:
%%time
for i in range(10,len(dax['Genres'].to_list()),10):
    inputt=dax['Genres'].to_list()[:i]
    data=[[i,process.extractBests(i, choices,score_cutoff=80,limit=20)] for i in inputt if i not in choices if i not in li]
    list_prep(data)

Wall time: 1min 30s


In [15]:
choices_final=[row['Genres'] for i,row in dax.iterrows() if row['Genres'] in choices if row['counts']>50]

In [16]:
choices_final

['',
 'drama',
 'comedy',
 'horror',
 'action',
 'thriller',
 'romance',
 'western',
 'crime',
 'animated',
 'adventure',
 'musical',
 'science fiction',
 ' noir',
 'mystery',
 'war',
 'sci-fi',
 'biography',
 'family',
 'fantasy',
 'anime',
 'social',
 'historical',
 'documentary',
 'serial',
 'martial_arts']

In [17]:

def genre_list(x):
    
    if x in choices_final:
        return x
    elif process.extractBests(x, choices_final,score_cutoff=80,limit=20) !=[]:
        a=process.extractBests(x, choices_final,score_cutoff=80,limit=20)
        return ','.join([a[i][0] for i in range(len(a))])
    else:
        return ''

In [18]:
genre_list('drama romance')

'drama,romance'

In [19]:
df['Genre_1']=df['Genre']

In [20]:
df['Genre_1']=df['Genre_1'].apply(lambda x: genre_list(x))

In [21]:
def genre(data):
    df1 = pd.DataFrame(columns = ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast','Genre', 'Wiki Page', 'Plot', 'Genre_1','Genre_updated'])
    for i,row in data.iterrows():
        if len(row['Genre_1'].split(','))>1:
            for i in row['Genre_1'].split(','):
               # print(i)
                df=pd.DataFrame(row).T
                df['Genre_updated']=i
                df1=pd.concat([df1,df])
        else:
            df=pd.DataFrame(row).T
            df['Genre_updated']=df['Genre_1']
            df1=pd.concat([df1,df])
            
    return df1.reset_index(drop=True)

In [22]:
%%time
new_df=genre(df)

Wall time: 2min 34s


In [23]:
new_df['Genre_updated'].value_counts()

drama              9432
comedy             7266
                   6777
action             2446
romance            1973
thriller           1916
crime              1597
horror             1576
western             974
musical             950
animated            894
adventure           868
family              634
war                 600
science fiction     582
fantasy             528
mystery             481
biography           457
sci-fi              389
 noir               356
historical          257
social              223
anime               143
documentary         130
martial_arts        129
serial               83
Name: Genre_updated, dtype: int64

In [ ]:
new_df.drop(['Genre_1'],axis=1).to_csv('wiki_movie_plots_deduped_cleaned_genre.csv',index=False)

In [ ]:
new_df[new_df['Genre']!=new_df['Genre_updated']]

In [ ]:
new_df[(new_df['Genre_updated']=='') & (new_df['Genre']!=new_df['Genre_updated'])]

In [ ]:
new_df[(new_df['Genre_updated']=='') & (new_df['Genre']!=new_df['Genre_updated'])]['Genre'].unique()